<a href="https://colab.research.google.com/github/gowtham-dd/Data-Science-Basics/blob/main/INJURY_RECOVERY_PREDICTION_USING_LINEAR_REGRESSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BREAST CANCER PREDICTION USING LINEAR REGRESSION

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/breast-cancer-dataset")

print("Path to dataset files:", path)

100%|██████████| 48.6k/48.6k [00:00<00:00, 53.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/yasserh/breast-cancer-dataset/versions/1


In [16]:
import pandas as pd

df=pd.read_csv('/content/injury_data.csv')
df

,Player_Age,Player_Weight,Player_Height,Previous_Injuries,Training_Intensity,Recovery_Time,Likelihood_of_Injury
0,24,66.251933,175.732429,1,0.457929,5,0
1,37,70.996271,174.581650,0,0.226522,6,1
2,32,80.093781,186.329618,0,0.613970,2,1
3,28,87.473271,175.504240,1,0.252858,4,1
4,25,84.659220,190.175012,0,0.577632,1,1
...,...,...,...,...,...,...,...
995,23,99.147910,165.290912,0,0.077857,3,1
996,23,75.799934,178.132281,1,0.955448,1,0
997,20,78.479061,173.823908,0,0.246961,1,1
998,24,66.915802,197.616889,1,0.859645,2,0


In [19]:
#glucose,ski,bp features
#bmi - target

features=df[['Player_Age','Player_Weight','Player_Height','Previous_Injuries','Training_Intensity','Likelihood_of_Injury']]
target=df['Recovery_Time']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(features,target,test_size=0.2,random_state=42)
#0.2 testing 20%

from sklearn.linear_model import LinearRegression
model=LinearRegression()

model.fit(X_train,y_train)
print(model.coef_)#m value
print(model.intercept_)#c inntercept / bias
print(model.predict([[24,66.251933,	175.732429,	1,	0.457929,0]]))
#y=mx+c

[-0.00851108 -0.01601189  0.00568199 -0.00703118 -0.27951822 -0.1147908 ]
4.084022677150119
[3.68241731]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [20]:
#Model Evaluation
#1. MAE - Model Absolute Error = avg(sum(Absolute(Predicted(y))))

model.predict(X_test)
print(model.score(X_test,y_test))
from sklearn.metrics import r2_score
r2_score(y_test,model.predict(X_test))
print(r2_score(y_test,model.predict(X_test)))


-0.0271360232541773
-0.0271360232541773


In [21]:

from sklearn import metrics
import numpy as np
y_pred=model.predict(X_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)


Mean Absolute Error: 1.4829607259069206
Mean Squared Error: 2.8753389050971627
Root Mean Squared Error: 1.69568243049728


-0.0271360232541773

In [23]:
import pickle

#save
filename='linear_regression_model.pkl'
pickle.dump(model,open(filename,'wb'))

#Load
loaded_model=pickle.load(open(filename,'rb'))


In [25]:
!pip install gradio matplotlib
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

# Load the pre-trained model
try:
    model = pkl.load(open('linear_regression_model.pkl', 'rb'))
except FileNotFoundError:
    print("Error: Model file 'linear_regression_model.pkl' not found.")
    model = None  # Prevents execution errors

def predict_recoverymonth(Player_Age, Player_Weight, Player_Height, Previous_Injuries, Training_Intensity, Likelihood_of_Injury):
    """Predicts Muscle Injury Recovery Time and generates a chart."""
    if model:
        prediction = model.predict([[Player_Age, Player_Weight, Player_Height, Previous_Injuries, Training_Intensity, Likelihood_of_Injury]])[0]

        # 📊 Generate a bar chart for recovery time
        factors = ["Age", "Weight", "Height", "Prev Injuries", "Training Intensity", "Likelihood of Injury"]
        values = [Player_Age, Player_Weight, Player_Height, Previous_Injuries, Training_Intensity, Likelihood_of_Injury]

        fig, ax = plt.subplots(figsize=(6, 4))
        ax.bar(factors, values, color=["blue", "green", "red", "purple", "orange", "cyan"])
        ax.set_ylabel("Feature Values")
        ax.set_title(f"Recovery Time Prediction: {round(prediction, 2)} months")
        plt.xticks(rotation=45)

        return round(prediction, 2), fig  # Returning both the predicted value and the chart

    else:
        return "Model not loaded!", None

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_recoverymonth,
    inputs=[
        gr.Number(label="Age"),
        gr.Number(label="Weight"),
        gr.Number(label="Height"),
        gr.Number(label="Previous Injuries"),
        gr.Number(label="Training Intensity"),
        gr.Number(label="Likelihood of Injury")
    ],
    outputs=[
        gr.Number(label="Recovery Month"),
        gr.Plot(label="Feature Impact on Recovery Time")  # Chart output
    ],
    title="Muscle Recovery Predictor with Chart Analysis",
    description="Enter the player details to predict recovery time and visualize the impact of different features."
)

# Launch the interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b38111b77a9be6a45e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
